In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  GridSearchCV,KFold

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [4]:
data=pd.read_csv("diabetes.csv")

In [5]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
data.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
Outcome                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [8]:
# Minimum of glucose,BMI,BP shows out to be 0 (missing data/outlier)
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [9]:
data[data["Glucose"]==0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [10]:
data[data["BMI"]==0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
9,8,125,96,0,0,0.0,0.232,54,1
49,7,105,0,0,0,0.0,0.305,24,0
60,2,84,0,0,0,0.0,0.304,21,0
81,2,74,0,0,0,0.0,0.102,22,0
145,0,102,75,23,0,0.0,0.572,21,0
371,0,118,64,23,89,0.0,1.731,21,0
426,0,94,0,0,0,0.0,0.256,25,0
494,3,80,0,0,0,0.0,0.174,22,0
522,6,114,0,0,0,0.0,0.189,26,0
684,5,136,82,0,0,0.0,0.640,69,0


In [11]:
print("Shape of Dataset: ",data.shape)

Shape of Dataset:  (768, 9)


# Preprocess Dataset

In [12]:
# Replace 0 for these columns by numpy Nan
cols=["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]

for c in cols:
    data[c].replace(0,np.NaN,inplace=True)

In [13]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [14]:
data.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [15]:
# Remove Null Values
data.dropna(inplace=True)
print("Shape of Dataset: ",data.shape)

Shape of Dataset:  (392, 9)


In [16]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


# Splitting Dataset

In [17]:
X=data.drop(["Outcome"],axis=1)
y=data["Outcome"]

In [18]:
print("Shape of X:",X.shape)
print("Shape of y:",y.shape)

Shape of X: (392, 8)
Shape of y: (392,)


In [19]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26
8,2,197.0,70.0,45.0,543.0,30.5,0.158,53
13,1,189.0,60.0,23.0,846.0,30.1,0.398,59


# Normalise the Scale

In [20]:
scaler=StandardScaler().fit(X)

In [21]:
X_std=scaler.transform(X)

Xfin=pd.DataFrame(X_std)

Xfin.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [22]:
Xfin.columns

RangeIndex(start=0, stop=8, step=1)

In [23]:
# function to define Model
def  model_create():
    #Create Model
    model=Sequential()
    model.add(Dense(8,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(4,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    #Compile The Model
    adam = Adam(lr =0.01)
    model.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
    return model

In [24]:
model=model_create()
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
# function to define Model
seed=6
np.random.seed(seed)
def  model_create_new():
    #Create Model
    model=Sequential()
    model.add(Dense(8,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(4,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    #Compile The Model
    adam = Adam(lr =0.01)
    model.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
    return model

model_new= KerasClassifier(build_fn= model_create_new,verbose=1)

# Define Grid Search

batch_size=[10,20,40]
epochs=[10,25,50]

# make a dictionary for gridSearch
param_grid=dict(batch_size=batch_size,epochs=epochs)

#Build The Grid Search
grid=GridSearchCV(estimator=model_new,param_grid=param_grid,cv=KFold(random_state=seed),verbose=5)

grid_results=grid.fit(Xfin,y)

# Summarise Results

print("Best Score {} , using {} ".format(grid_results.best_score_,grid_results.best_params_))
mean=grid_results.cv_results_['mean_test_score']
stds=grid_results.cv_results_['std_test_score']
params=grid_results.cv_results_['params']

for mean,stdev,params in zip(mean,stds,params):
    print("{} , {} with : {} ".format(mean,stdev,params))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/10
261/261 [==============================] - 11s 42ms/step - loss: 0.6309 - acc: 0.6935
Epoch 2/10
261/261 [==============================] - 0s 774us/step - loss: 0.4833 - acc: 0.6973
Epoch 3/10
261/261 [==============================] - 0s 754us/step - loss: 0.4416 - acc: 0.6973
Epoch 4/10
261/261 [==============================] - 0s 742us/step - loss: 0.4250 - acc: 0.6973
Epoch 5/10
261/261 [==============================] - 0s 798us/step - loss: 0.4142 - acc: 0.8123
Epoch 6/10
261/261 [==============================] - 0s 756us/step - loss: 0.4001 - acc: 0.8429
Epoch 7/10
261/261 [==============================] - 0s 744us/step - loss: 0.3908 - acc: 0.8314
Epoch 8/10
261/261 [==============================] - 0s 761us/step - loss: 0.3876 - acc: 0.8314
Epoch 9/10
261/261 [==============================] - 0s 740us/step - loss: 0.3820 - acc: 0.8352
Epoch 10/10
131/131 [==============================] - 4s 31ms/step
[CV] ............ batch_size=10, epochs=10, score=0.725, to

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.6s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 11s 42ms/step - loss: 0.6399 - acc: 0.6475
Epoch 2/10
261/261 [==============================] - 0s 801us/step - loss: 0.5227 - acc: 0.6552
Epoch 3/10
261/261 [==============================] - 0s 828us/step - loss: 0.4920 - acc: 0.6552
Epoch 4/10
261/261 [==============================] - 0s 779us/step - loss: 0.4815 - acc: 0.6973
Epoch 5/10
261/261 [==============================] - 0s 802us/step - loss: 0.4742 - acc: 0.7854
Epoch 6/10
261/261 [==============================] - 0s 747us/step - loss: 0.4608 - acc: 0.8008
Epoch 7/10
261/261 [==============================] - 0s 712us/step - loss: 0.4544 - acc: 0.8008
Epoch 8/10
261/261 [==============================] - 0s 749us/step - loss: 0.4463 - acc: 0.8008
Epoch 9/10
261/261 [==============================] - 0s 738us/step - loss: 0.4350 - acc: 0.8046
Epoch 10/10
131/131 [==============================] - 4s 32ms/step
[CV] ............ batch_size=10, epochs=10, score=0.763, to

...

262/262 [==============================] - 0s 201us/step - loss: 0.5634 - acc: 0.7672
Epoch 11/25
262/262 [==============================] - 0s 196us/step - loss: 0.5526 - acc: 0.7595
Epoch 12/25
262/262 [==============================] - 0s 196us/step - loss: 0.5442 - acc: 0.7595
Epoch 13/25
262/262 [==============================] - 0s 234us/step - loss: 0.5364 - acc: 0.7634
Epoch 14/25
262/262 [==============================] - 0s 195us/step - loss: 0.5262 - acc: 0.7786
Epoch 15/25
262/262 [==============================] - 0s 200us/step - loss: 0.5217 - acc: 0.7939
Epoch 16/25
262/262 [==============================] - 0s 210us/step - loss: 0.5166 - acc: 0.7748
Epoch 17/25
262/262 [==============================] - 0s 194us/step - loss: 0.5059 - acc: 0.7824
Epoch 18/25
262/262 [==============================] - 0s 207us/step - loss: 0.5019 - acc: 0.7901
Epoch 19/25
262/262 [==============================] - 0s 228us/step - loss: 0.4955 - acc: 0.7863
Epoch 20/25
262/262 [===========

261/261 [==============================] - 0s 213us/step - loss: 0.4135 - acc: 0.8161
Epoch 15/50
261/261 [==============================] - 0s 212us/step - loss: 0.4102 - acc: 0.8123
Epoch 16/50
261/261 [==============================] - 0s 212us/step - loss: 0.4095 - acc: 0.8276
Epoch 17/50
261/261 [==============================] - 0s 198us/step - loss: 0.4030 - acc: 0.8276
Epoch 18/50
261/261 [==============================] - 0s 202us/step - loss: 0.4053 - acc: 0.8161
Epoch 19/50
261/261 [==============================] - 0s 193us/step - loss: 0.4027 - acc: 0.8199
Epoch 20/50
261/261 [==============================] - 0s 199us/step - loss: 0.4020 - acc: 0.8199
Epoch 21/50
261/261 [==============================] - 0s 199us/step - loss: 0.3969 - acc: 0.8238
Epoch 22/50
261/261 [==============================] - 0s 179us/step - loss: 0.3961 - acc: 0.8238
Epoch 23/50
261/261 [==============================] - 0s 187us/step - loss: 0.3945 - acc: 0.8161
Epoch 24/50
261/261 [===========

262/262 [==============================] - 0s 201us/step - loss: 0.3454 - acc: 0.8473
Epoch 46/50
262/262 [==============================] - 0s 198us/step - loss: 0.3494 - acc: 0.8550
Epoch 47/50
262/262 [==============================] - 0s 196us/step - loss: 0.3478 - acc: 0.8397
Epoch 48/50
262/262 [==============================] - 0s 210us/step - loss: 0.3401 - acc: 0.8473
Epoch 49/50
262/262 [==============================] - 0s 209us/step - loss: 0.3424 - acc: 0.8511
Epoch 50/50
130/130 [==============================] - 5s 41ms/step
[CV] ............ batch_size=40, epochs=50, score=0.838, total=  22.7s


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  9.7min finished


Epoch 1/10
392/392 [==============================] - 14s 35ms/step - loss: 0.6355 - acc: 0.6709
Epoch 2/10
392/392 [==============================] - 0s 390us/step - loss: 0.4985 - acc: 0.7628
Epoch 3/10
392/392 [==============================] - 0s 421us/step - loss: 0.4515 - acc: 0.7653
Epoch 4/10
392/392 [==============================] - 0s 368us/step - loss: 0.4422 - acc: 0.7883
Epoch 5/10
392/392 [==============================] - 0s 369us/step - loss: 0.4420 - acc: 0.7755
Epoch 6/10
392/392 [==============================] - 0s 369us/step - loss: 0.4320 - acc: 0.7985
Epoch 7/10
392/392 [==============================] - 0s 366us/step - loss: 0.4278 - acc: 0.7908
Epoch 8/10
392/392 [==============================] - 0s 376us/step - loss: 0.4270 - acc: 0.7985
Epoch 9/10
392/392 [==============================] - 0s 373us/step - loss: 0.4230 - acc: 0.7959
Epoch 10/10
392/392 [==============================] - 0s 401us/step - loss: 0.4277 - acc: 0.8087
Best Score 0.7857142898197077